In [1]:
import os
import numpy as np
import pandas as pd
from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib.pyplot as plt
from hough import *
from conformal_map import *
from tqdm import tqdm

%matplotlib inline

In [2]:
B = 1000
def convertTrackToImage(trackId):
    first = True
    imageBins = B
    image = np.empty((imageBins, 0))
    for volume in [8, 13, 17]:
        #print(volume)
        for layer in hits[hits.volume_id == volume].layer_id.unique():
            #print(layer)
            truthVol8 = truth.iloc[hits.index[(hits.volume_id == volume)].tolist()]
            hitsVol8 = hits.iloc[hits.index[(hits.volume_id == volume)].tolist()]
            t = truthVol8.loc[(hitsVol8.layer_id == layer) & (truthVol8.particle_id == trackId)]
            r = np.power(t.tx, 2) + np.power(t.ty, 2)
            theta = np.arctan(t.ty / t.tx)
            #print(theta)
            thetaRange = np.pi * 2
            indicesFloat = (theta / thetaRange) * imageBins + imageBins / 2
            #print(indicesFloat)
            indices = np.array(list(map(int, indicesFloat)))
            indices = np.reshape(indices, (len(indices), 1))
            bins = np.zeros((imageBins, 1))
            if first:
                if len(indices) == 0:
                    image = np.hstack([image, np.zeros((imageBins, 1))])
                    continue
                firstIndex = max(indices[0])
                moveRange = firstIndex - imageBins // 2
                first = False
            indices = (indices - moveRange) % imageBins
            for i, index in enumerate(indices):
                bins[index, 0] += 1
            image = np.hstack([image, bins])
            #print(indices)
    return image

In [3]:
images = []
train_y = np.empty((0, 2))
train_y_mag = np.empty((0, 1))

In [ ]:
for k in range(5):
    print("round:", k)
    #if k < 10:
      #  event_prefix = 'event00000101' + str(k)
    #else:
     #   event_prefix = 'event00000103' + str(k - 10)
    event_prefix = 'event00000104' + str(k)
    hits, cells, particles, truth = load_event(os.path.join('train_100_events', event_prefix))
    cond = (hits['volume_id'] == 8) | (hits['volume_id'] == 13) | (hits['volume_id'] == 17)
    selected_indices = hits.index[cond].tolist()
    selected_hits = hits.iloc[selected_indices]
    selected_truth = truth.iloc[selected_indices]
    # Create training set with tracks that are minimum of 5 hits
    tracks = selected_truth.particle_id.unique()
    validTracks = []
    for track in tracks:
        if track == 0:
            continue
        numberHits = selected_truth[selected_truth.particle_id == track].shape[0]
        t = selected_truth[selected_truth.particle_id == track]
        if numberHits >= 11:
            validTracks.append(track)

    nr = len(validTracks)
    for j in tqdm(range(nr)):
        trackId = validTracks[j]
        image = convertTrackToImage(trackId)
        images.append(image)
        #print(c)
    # Create train_Y (the momentum)
    for j, track in enumerate(validTracks[0:nr]):
        track = validTracks[j]
        if track == 0:
            continue
        t = selected_truth[selected_truth.particle_id == track]
        px = t['tpx'].mean()
        py = t['tpy'].mean()
        p = np.sqrt(px**2 + py**2)
        train_y = np.vstack([train_y, [px, py]])
        train_y_mag = np.vstack([train_y_mag, [p]])

In [ ]:
images_val = []
train_y_val = np.empty((0, 2))
train_y_val_mag = np.empty((0, 1))

In [ ]:
for k in range(0, 10):
    print("round:", k)
    event_prefix = 'event00000102' + str(k)
    hits, cells, particles, truth = load_event(os.path.join('train_100_events', event_prefix))
    cond = (hits['volume_id'] == 8) | (hits['volume_id'] == 13) | (hits['volume_id'] == 17)
    selected_indices = hits.index[cond].tolist()
    selected_hits = hits.iloc[selected_indices]
    selected_truth = truth.iloc[selected_indices]
    # Create training set with tracks that are minimum of 5 hits
    tracks = selected_truth.particle_id.unique()
    validTracks = []
    for track in tracks:
        if track == 0:
            continue
        numberHits = selected_truth[selected_truth.particle_id == track].shape[0]
        t = selected_truth[selected_truth.particle_id == track]
        if numberHits >= 12:
            validTracks.append(track)

    nr = len(validTracks)
    for j in tqdm(range(nr)):
        trackId = validTracks[j]
        image = convertTrackToImage(trackId)
        images_val.append(image)
        #print(c)
    # Create train_Y (the momentum)
    for j, track in enumerate(validTracks[0:nr]):
        if track == 0:
            continue
        t = selected_truth[selected_truth.particle_id == track]
        px = t['tpx'].mean()
        py = t['tpy'].mean()
        #pz = t['tpz'].mean()
        p = np.sqrt(px**2 + py**2) #+ pz**2)# / (0.3 * 0.001 * 2)
        train_y_val_mag = np.vstack([train_y_val_mag, [p]])
        train_y_val = np.vstack([train_y_val, [px, py]])

In [ ]:
#train_y = np.reshape(train_y, (len(train_y), 2))
#train_y_mag = np.reshape(train_y_mag, (len(train_y_mag), 1))
#train_X = np.reshape(images, (len(images), B, 10, 1))

#test_X = np.reshape(images_val, (len(images_val), B, 10, 1))
#test_y = np.reshape(train_y_val, (len(train_y_val), 2))
#test_y_mag = np.reshape(train_y_val_mag, (len(train_y_val_mag), 1))

In [ ]:
#np.savez("", X=train_X, y=train_y, y_mag=train_y_mag, X_test=test_X, y_mag_test=test_y_mag, y_test=test_y)